In [1]:

# imports
import os
import sys
import types
import json
import base64

# figure size/format
fig_width = 7
fig_height = 5
fig_format = 'retina'
fig_dpi = 96
interactivity = ''
is_shiny = False
is_dashboard = False
plotly_connected = True

# matplotlib defaults / format
try:
  import matplotlib.pyplot as plt
  plt.rcParams['figure.figsize'] = (fig_width, fig_height)
  plt.rcParams['figure.dpi'] = fig_dpi
  plt.rcParams['savefig.dpi'] = "figure"
  from IPython.display import set_matplotlib_formats
  set_matplotlib_formats(fig_format)
except Exception:
  pass

# plotly use connected mode
try:
  import plotly.io as pio
  if plotly_connected:
    pio.renderers.default = "notebook_connected"
  else:
    pio.renderers.default = "notebook"
  for template in pio.templates.keys():
    pio.templates[template].layout.margin = dict(t=30,r=0,b=0,l=0)
except Exception:
  pass

# disable itables paging for dashboards
if is_dashboard:
  try:
    from itables import options
    options.dom = 'fiBrtlp'
    options.maxBytes = 1024 * 1024
    options.language = dict(info = "Showing _TOTAL_ entries")
    options.classes = "display nowrap compact"
    options.paging = False
    options.searching = True
    options.ordering = True
    options.info = True
    options.lengthChange = False
    options.autoWidth = False
    options.responsive = True
    options.keys = True
    options.buttons = []
  except Exception:
    pass
  
  try:
    import altair as alt
    # By default, dashboards will have container sized
    # vega visualizations which allows them to flow reasonably
    theme_sentinel = '_quarto-dashboard-internal'
    def make_theme(name):
        nonTheme = alt.themes._plugins[name]    
        def patch_theme(*args, **kwargs):
            existingTheme = nonTheme()
            if 'height' not in existingTheme:
              existingTheme['height'] = 'container'
            if 'width' not in existingTheme:
              existingTheme['width'] = 'container'

            if 'config' not in existingTheme:
              existingTheme['config'] = dict()
            
            # Configure the default font sizes
            title_font_size = 15
            header_font_size = 13
            axis_font_size = 12
            legend_font_size = 12
            mark_font_size = 12
            tooltip = False

            config = existingTheme['config']

            # The Axis
            if 'axis' not in config:
              config['axis'] = dict()
            axis = config['axis']
            if 'labelFontSize' not in axis:
              axis['labelFontSize'] = axis_font_size
            if 'titleFontSize' not in axis:
              axis['titleFontSize'] = axis_font_size  

            # The legend
            if 'legend' not in config:
              config['legend'] = dict()
            legend = config['legend']
            if 'labelFontSize' not in legend:
              legend['labelFontSize'] = legend_font_size
            if 'titleFontSize' not in legend:
              legend['titleFontSize'] = legend_font_size  

            # The header
            if 'header' not in config:
              config['header'] = dict()
            header = config['header']
            if 'labelFontSize' not in header:
              header['labelFontSize'] = header_font_size
            if 'titleFontSize' not in header:
              header['titleFontSize'] = header_font_size    

            # Title
            if 'title' not in config:
              config['title'] = dict()
            title = config['title']
            if 'fontSize' not in title:
              title['fontSize'] = title_font_size

            # Marks
            if 'mark' not in config:
              config['mark'] = dict()
            mark = config['mark']
            if 'fontSize' not in mark:
              mark['fontSize'] = mark_font_size

            # Mark tooltips
            if tooltip and 'tooltip' not in mark:
              mark['tooltip'] = dict(content="encoding")

            return existingTheme
            
        return patch_theme

    # We can only do this once per session
    if theme_sentinel not in alt.themes.names():
      for name in alt.themes.names():
        alt.themes.register(name, make_theme(name))
      
      # register a sentinel theme so we only do this once
      alt.themes.register(theme_sentinel, make_theme('default'))
      alt.themes.enable('default')

  except Exception:
    pass

# enable pandas latex repr when targeting pdfs
try:
  import pandas as pd
  if fig_format == 'pdf':
    pd.set_option('display.latex.repr', True)
except Exception:
  pass

# interactivity
if interactivity:
  from IPython.core.interactiveshell import InteractiveShell
  InteractiveShell.ast_node_interactivity = interactivity

# NOTE: the kernel_deps code is repeated in the cleanup.py file
# (we can't easily share this code b/c of the way it is run).
# If you edit this code also edit the same code in cleanup.py!

# output kernel dependencies
kernel_deps = dict()
for module in list(sys.modules.values()):
  # Some modules play games with sys.modules (e.g. email/__init__.py
  # in the standard library), and occasionally this can cause strange
  # failures in getattr.  Just ignore anything that's not an ordinary
  # module.
  if not isinstance(module, types.ModuleType):
    continue
  path = getattr(module, "__file__", None)
  if not path:
    continue
  if path.endswith(".pyc") or path.endswith(".pyo"):
    path = path[:-1]
  if not os.path.exists(path):
    continue
  kernel_deps[path] = os.stat(path).st_mtime
print(json.dumps(kernel_deps))

# set run_path if requested
run_path = 'L1VzZXJzL3Nrb2pha3UtYWRtaW4vRG9jdW1lbnRzL3Byb2plY3RzL2FwcGxpZWQtc29mdC1jb21wL2RvY3MvbGVjdHVyZS1ub3RlL20wMy10ZXh0'
if run_path:
  # hex-decode the path
  run_path = base64.b64decode(run_path.encode("utf-8")).decode("utf-8")
  os.chdir(run_path)

# reset state
%reset

# shiny
# Checking for shiny by using False directly because we're after the %reset. We don't want
# to set a variable that stays in global scope.
if False:
  try:
    import htmltools as _htmltools
    import ast as _ast

    _htmltools.html_dependency_render_mode = "json"

    # This decorator will be added to all function definitions
    def _display_if_has_repr_html(x):
      try:
        # IPython 7.14 preferred import
        from IPython.display import display, HTML
      except:
        from IPython.core.display import display, HTML

      if hasattr(x, '_repr_html_'):
        display(HTML(x._repr_html_()))
      return x

    # ideally we would undo the call to ast_transformers.append
    # at the end of this block whenver an error occurs, we do 
    # this for now as it will only be a problem if the user 
    # switches from shiny to not-shiny mode (and even then likely
    # won't matter)
    import builtins
    builtins._display_if_has_repr_html = _display_if_has_repr_html

    class _FunctionDefReprHtml(_ast.NodeTransformer):
      def visit_FunctionDef(self, node):
        node.decorator_list.insert(
          0,
          _ast.Name(id="_display_if_has_repr_html", ctx=_ast.Load())
        )
        return node

      def visit_AsyncFunctionDef(self, node):
        node.decorator_list.insert(
          0,
          _ast.Name(id="_display_if_has_repr_html", ctx=_ast.Load())
        )
        return node

    ip = get_ipython()
    ip.ast_transformers.append(_FunctionDefReprHtml())

  except:
    pass

def ojs_define(**kwargs):
  import json
  try:
    # IPython 7.14 preferred import
    from IPython.display import display, HTML
  except:
    from IPython.core.display import display, HTML

  # do some minor magic for convenience when handling pandas
  # dataframes
  def convert(v):
    try:
      import pandas as pd
    except ModuleNotFoundError: # don't do the magic when pandas is not available
      return v
    if type(v) == pd.Series:
      v = pd.DataFrame(v)
    if type(v) == pd.DataFrame:
      j = json.loads(v.T.to_json(orient='split'))
      return dict((k,v) for (k,v) in zip(j["index"], j["data"]))
    else:
      return v

  v = dict(contents=list(dict(name=key, value=convert(value)) for (key, value) in kwargs.items()))
  display(HTML('<script type="ojs-define">' + json.dumps(v) + '</script>'), metadata=dict(ojs_define = True))
globals()["ojs_define"] = ojs_define
globals()["__spec__"] = None

/var/folders/j7/9dgqq5g53vnbsbmvh2yqtckr0000gr/T/ipykernel_47382/3548943389.py:25: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats(fig_format)


{"/Users/skojaku-admin/miniforge3/envs/applsoftcomp/lib/python3.10/importlib/_bootstrap.py": 1703345986.9647622, "/Users/skojaku-admin/miniforge3/envs/applsoftcomp/lib/python3.10/importlib/_bootstrap_external.py": 1703345986.9687936, "/Users/skojaku-admin/miniforge3/envs/applsoftcomp/lib/python3.10/codecs.py": 1703345984.5304527, "/Users/skojaku-admin/miniforge3/envs/applsoftcomp/lib/python3.10/encodings/aliases.py": 1703345985.7136338, "/Users/skojaku-admin/miniforge3/envs/applsoftcomp/lib/python3.10/encodings/__init__.py": 1703345985.707323, "/Users/skojaku-admin/miniforge3/envs/applsoftcomp/lib/python3.10/encodings/utf_8.py": 1703345986.387034, "/Users/skojaku-admin/miniforge3/envs/applsoftcomp/lib/python3.10/abc.py": 1703345984.4232924, "/Users/skojaku-admin/miniforge3/envs/applsoftcomp/lib/python3.10/io.py": 1703345984.6479, "/Users/skojaku-admin/miniforge3/envs/applsoftcomp/lib/python3.10/stat.py": 1703345984.8209362, "/Users/skojaku-admin/miniforge3/envs/applsoftcomp/lib/python3

In [2]:
from transformers import AutoTokenizer
import os

model_name = "microsoft/phi-1.5"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [3]:
#| code-fold: true
print(f"Vocabulary size: {tokenizer.vocab_size:,} tokens")
print(f"Max sequence length: {tokenizer.model_max_length} tokens")

Vocabulary size: 50,257 tokens
Max sequence length: 2048 tokens


In [4]:
text = "Binghamton University."

tokens = tokenizer.tokenize(text) # Tokenize the text

In [5]:
#| code-fold: true
print(f"Tokens: {tokens}")

Tokens: ['B', 'ingham', 'ton', 'ĠUniversity', '.']


In [6]:
#| code-fold: true

texts = [
    "Bearcats",
    "New York",
]

print("Word tokenization examples:\n")
for text in texts:
    tokens = tokenizer.tokenize(text)
    print(f"{text:10s} → {tokens}")

Word tokenization examples:

Bearcats   → ['Bear', 'cats']
New York   → ['New', 'ĠYork']


In [7]:
#| code-fold: true

text = "Binghamton University"

# Get token IDs
token_ids = tokenizer.encode(text, add_special_tokens=False)
tokens = tokenizer.tokenize(text)

print("Token → Token ID mapping:\n")
for token, token_id in zip(tokens, token_ids):
    print(f"{token:10s} → {token_id:6d}")

Token → Token ID mapping:

B          →     33
ingham     →  25875
ton        →   1122
ĠUniversity →   2059


In [8]:
# Get the full vocabulary
vocab = tokenizer.get_vocab()

# Sample some tokens
sample_tokens = list(vocab.items())[:5]
for token, id in sample_tokens:
    print(f"  {id:6d}: '{token}'")

   34722: 'Ġchew'
   22536: 'Ġambulance'
   39037: 'ĠSurgery'
   36583: 'Ġdivert'
   30936: 'ĠWORK'


In [9]:
token_id = [50256] # the token ID of <|endoftext|>
token = tokenizer.convert_ids_to_tokens(token_id)[0]
print(f"Token ID: {token_id} → Token: {token}")

Token ID: [50256] → Token: <|endoftext|>


In [10]:
from transformers import AutoModelForCausalLM
import torch

# load the model
model = AutoModelForCausalLM.from_pretrained(model_name)

# Retrieve the embedding layer
embedding_layer = model.model.embed_tokens

In [11]:
#| code-fold: true
print(embedding_layer.weight[:5, :10])

tensor([[ 0.0097, -0.0155,  0.0603,  0.0326, -0.0108, -0.0271, -0.0273,  0.0178,
         -0.0242,  0.0100],
        [ 0.0243,  0.0543,  0.0178, -0.0679, -0.0130,  0.0265, -0.0423, -0.0287,
         -0.0051, -0.0179],
        [-0.0416,  0.0370, -0.0160, -0.0254, -0.0371, -0.0208, -0.0023,  0.0647,
          0.0468,  0.0013],
        [-0.0051, -0.0044,  0.0248, -0.0489,  0.0399,  0.0005, -0.0070,  0.0148,
          0.0030,  0.0070],
        [ 0.0289,  0.0362, -0.0027, -0.0775, -0.0136, -0.0203, -0.0566, -0.0558,
          0.0269, -0.0067]], grad_fn=<SliceBackward0>)
